# Batch processing
Tobias Rose 2020

# Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
import sys
import os
from pathlib import Path
from scipy.signal import convolve
#from skimage.viewer import ImageViewer
from tqdm import tqdm

### User-specific folders

In [ ]:
if sys.platform == "darwin":
    ### Mac
    main_root = '/Volumes/archive_bonhoeffer_group$/David Laubender/Data/imaging data/DL_191024_6/ImagingData/' #location of original data
    adata     = '/Volumes/archive_bonhoeffer_group$/David Laubender/adata' #location of saved analyzed data
    ftemp     = '/Users/trose/Data/temp' #fast disk (local ssd for s2p binary files) 
    ftiff     = '/Users/trose/Data/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)
elif sys.platform == "win32":
    main_root = 'I:/David Laubender/Data/imaging data/DL_191024_6/ImagingData' #location of original data
    adata     = 'I:/David Laubender/adata' #location of saved analyzed data
    ftemp     = 'C:/temp/trose/suite2ptemp' #fast disk (local ssd for s2p binary files  
    ftiff     = 'C:/temp/trose/s2p_tiff' #fast disk folder for concatenated tiffs (if needed)

In [ ]:
exp = ['62283', '62284', '62285', '62286', '62287', '62288', '62289', '62290']

In [ ]:
exp = ['62282']

In [ ]:
runops = {
    'exp': exp, 
    'concat': False,               #whether or not to run all experiments as a concatenated single experiment
    'main_root': main_root,
    'adata': adata,
    'darkframes': 50,              #number of darkframes to be excluded from extraction
    'readfiles':10
}

In [ ]:
adata_s2ppath = []
adata_roipath = []
for val in exp:
    files = list(Path(main_root).rglob('suite2p_exp'+val+'/')) #recursive
    print(files)
    try:
        adata_s2ppath.append(os.path.join(files[0], 'suite2p', 'combined'))
        adata_roipath.append(os.path.dirname(files[0]))
    except:
        print(val + ' not found')
        
adata_s2ppath = sorted(adata_s2ppath)

In [ ]:
os.path.join(os.path.dirname(files[0]), 'Pixelmaps','exp'+ val,'templates', 'ROIs')

In [ ]:
adata_roipath

In [ ]:
froot = os.path.join(adata_roipath[0], 'Pixelmaps','exp'+ val,'templates', 'ROIs')
ftiffroot = os.path.join(adata_roipath[0], 'Pixelmaps','exp'+ val,'templates')
ftiffmean = Path(adata_s2ppath[0]).parts[-6] + '_' + Path(adata_s2ppath[0]).parts[-4] + '_exp' + val + '_s2p_avg_green.tif'
ftiffcorr = Path(adata_s2ppath[0]).parts[-6] + '_' + Path(adata_s2ppath[0]).parts[-4] + '_exp' + val + '_s2p_corr_green.tif'
adataname = os.path.join(adata, Path(adata_s2ppath[0]).parts[-6] , Path(adata_s2ppath[0]).parts[-4], Path(adata_s2ppath[0]).parts[-6] + '-' + 's2p_Adata' + '-' + val + '.mat')


In [ ]:
froot

In [ ]:
os.path.join(adata, Path(adata_s2ppath[0]).parts[-6] + '-' + 'Adata' + '-' + val + '.mat')

In [ ]:
Path(adata_s2ppath[0]).parts

In [ ]:
adataname = os.path.join(adata, Path(adata_s2ppath[0]).parts[-6] , Path(adata_s2ppath[0]).parts[-4], Path(adata_s2ppath[0]).parts[-6] + '-' + 's2p_Adata' + '-' + val + '.mat')
print(adataname)

In [ ]:
F = np.load(os.path.join(adata_s2ppath[0],'F.npy'))
Fneu = np.load(os.path.join(adata_s2ppath[0],'Fneu.npy'))
spks = np.load(os.path.join(adata_s2ppath[0],'spks.npy'))
stat = np.load(os.path.join(adata_s2ppath[0],'stat.npy'), allow_pickle=True)
ops = np.load(os.path.join(adata_s2ppath[0],'ops.npy'), allow_pickle=True).item()
iscell = np.load(os.path.join(adata_s2ppath[0],'iscell.npy'))

In [ ]:
ops

### plot examples

In [ ]:
# show cells
im = np.zeros((ops['Ly'], ops['Lx']))
ncells = len(stat)

for n in range(0,ncells):
    ypix = stat[n]['ypix'][~stat[n]['overlap']]
    xpix = stat[n]['xpix'][~stat[n]['overlap']]
    im[ypix,xpix] = n+1
fig = plt.figure(figsize=(7,4))
plt.imshow(im)
plt.tight_layout()
plt.show()

In [ ]:
ops['tau'] = .7

In [ ]:
# show mean image
fig = plt.figure(figsize=(8,3))
plt.subplot(1,3,1)
plt.imshow(ops["meanImg"])
plt.subplot(1,3,2)
plt.imshow(ops["meanImgE"])
plt.subplot(1,3,3)
plt.imshow(ops["Vcorr"])
plt.show

In [ ]:
ops['tau'] = 0.5
bouton = 11;

In [ ]:
efilt = np.exp(- np.linspace(0,50,200) / (ops['tau'] * ops['fs']))
#efilt /= efilt.sum()
sout = convolve(spks[bouton,:], efilt)
sout = sout[:spks.shape[1]]

In [ ]:
fig = plt.figure(figsize=(16,4))
plt.plot(F[bouton]-Fneu[bouton] * .7)
plt.show

In [ ]:
plt.plot(sout)
plt.tight_layout()
plt.show
#plt.plot(F[10:])

In [ ]:
plt.plot(spks[11])
plt.show

In [ ]:
plt.figure(figsize=(7,4))
plt.imshow(spks[:100, :5000], vmax = 3, vmin = -0.5, aspect='auto', cmap = 'gray_r')
plt.title('sample of the neural data matrix')
plt.ylabel('boutons') 
plt.xlabel('time [samples]')

In [ ]:
ops

# ToDo

- batch run different deconvolution settings using this code snippet

In [ ]:
# compute deconvolution
from suite2p import dcnv
import numpy as np

tau = 1.0 # timescale of indicator
fs = 30.0 # sampling rate in Hz
neucoeff = 0.7 # neuropil coefficient
# for computing and subtracting baseline
baseline = 'maximin' # take the running max of the running min after smoothing with gaussian
sig_baseline = 10.0 # in bins, standard deviation of gaussian with which to smooth
win_baseline = 60.0 # in seconds, window in which to compute max/min filters

ops = {'tau': tau, 'fs': fs, 'neucoeff': neucoeff,
       'baseline': baseline, 'sig_baseline': sig_baseline, 'win_baseline': win_baseline}

# load traces and subtract neuropil
F = np.load('F.npy')
Fneu = np.load('Fneu.npy')
Fc = F - ops['neucoeff'] * Fneu

# baseline operation
Fc = dcnv.preprocess(Fc, ops)

# get spikes
spks = dcnv.oasis(Fc, ops)